# Modelling Weeks 2025

## Performing a Deep Analysis of a Linear Antenna

### Plotting the Radiation Pattern, Number of Bins, and Radiating Elements

**Authors**: Louis Salanon, Shihab Berel, Hedi Komiha  
**Date**: 05/02/2025  

---

### **Project Overview**

This project focuses on optimizing the performance of a linear antenna array by analyzing radiation patterns, determining the optimal number of bins and radiating elements, and evaluating beam steering techniques. The methodology involves various steps, each addressing specific aspects of the antenna's design and performance.

---

### **Steps Overview**

1. **Radiation Pattern Analysis**:
   - Analyze the radiation pattern for different windowing methods (Uniform, Hamming, Hann).
   - Visualize the effects of tapering techniques on the main lobe width and sidelobe suppression.

2. **Optimization of Bins**:
   - Calculate the number of angular bins needed to cover the desired angular range.
   - Ensure the spacing condition is satisfied.

3. **Scanning Time and Repetition Rate**:
   - Compute the total scanning time for a given target distance.
   - Determine the maximum repetition rate based on optimal values.

---

### **How to Use This Notebook**

- Each section is labeled as **Step I.J**, where **I** represents the step, and **J** indicates the sub-step or function.
- For example, **Step 1.1** refers to the first function in Step 1.
- Code cells are designed to be interactive, allowing you to run the computations directly.

---

### **Authors' Contributions**

- **Louis Salanon**: Analysis and implementation of radiation patterns.
- **Shihab Berel**: Optimization of bins and antenna configuration.
- **Hedi Komiha**: Documentation and overall project management.

---

### **Call to Action**

Feel free to explore the code, test the functions, and contribute to the project by suggesting improvements or sharing your insights!


Main libraries imports:

In [29]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.widgets import Slider, TextBox

3) (Step 3) The following cells showcase the code that will plot the radiation pattern, varying parameters, such as the phaseshift, minimum and 
maximum angle, etc. It essentially shows the core functions that achieve this result.

3.1) The following function computes the pattern's beamwidth, see report for a detailed explanation.

In [30]:
def snippet_compute_beamwidth(N, d, wavelength, delta_phi_rad):
    k = 2.0 * np.pi / wavelength
    phi_deg = np.linspace(-90, 90, 4000)
    phi_rad = np.radians(phi_deg)           
   
    # Compute the array factor for all angles at once via broadcasting.
    n = np.arange(N)[:, None]               
    beta = k * d * np.sin(phi_rad) + delta_phi_rad  
    I = np.sum(np.exp(1j * n * beta), axis=0)
   
    # Normalize the power pattern.
    pattern = np.abs(I)**2
    pattern /= (np.max(pattern) + 1e-12)
   
    # Find indices where the pattern exceeds 0.5.
    half_val = 0.5
    idx = np.where(pattern >= half_val)[0]
   
    if len(idx) > 1:
        bw_deg = phi_deg[idx[-1]] - phi_deg[idx[0]]
    else:
        bw_deg = 0.0
       
    return bw_deg

3.2) The following function is defined to compute the pattern characteristics such as the main lobe gain, its direction, the secondary lobe gain, etc. 

In [31]:
def compute_pattern_full(N=40, freq_hz=1.2e9, delta_phi=0.0, array_len=4.0, do_square=True):
    c = 3e8
    lam = c / freq_hz
    k = 2.0 * np.pi / lam
    d = array_len / (N - 1) if N > 1 else 0.0

    full_phi_deg = np.linspace(-90, 90, 4000)
    full_phi_rad = np.radians(full_phi_deg)

    I = np.zeros_like(full_phi_rad, dtype=complex)
    for i, phi in enumerate(full_phi_rad):
        beta = k * d * np.sin(phi) + delta_phi
        I[i] = np.sum(np.exp(1j * np.arange(N) * beta))

    pattern = np.abs(I)**2
    pattern /= (np.max(pattern) + 1e-12)

    # Determine HPBW based on half-power threshold.
    half_val = 0.5
    idx = np.where(pattern >= half_val)[0]
    if len(idx) > 1:
        hpbw = full_phi_deg[idx[-1]] - full_phi_deg[idx[0]]
        left_edge = full_phi_deg[idx[0]]
        right_edge = full_phi_deg[idx[-1]]
    else:
        hpbw = 0.0
        left_edge = 0.0
        right_edge = 0.0

    # Main beam direction is the angle of maximum normalized power.
    peak_index = np.argmax(pattern)
    main_beam_direction = full_phi_deg[peak_index]

    # Define a margin (in degrees) around the main lobe to exclude.
    margin = 2.0  
    sec_mask = (full_phi_deg < (left_edge - margin)) | (full_phi_deg > (right_edge + margin))
    sec_indices = np.where(sec_mask)[0]
    if len(sec_indices) > 0:
        secondary_index = sec_indices[np.argmax(pattern[sec_indices])]
        secondary_gain = pattern[secondary_index]
        secondary_angle = full_phi_deg[secondary_index]
    else:
        secondary_gain = 0.0
        secondary_angle = 0.0

    # Optionally square-root pattern if do_square is False (not used here).
    if not do_square:
        pattern = np.sqrt(pattern)

    return (full_phi_deg, pattern, hpbw, left_edge, right_edge,
            main_beam_direction, secondary_gain, secondary_angle)

3.3) The following code defines the LinearArrayApp GUI, to plot radiation pattern as a function of different parameters (condensed because not really interesting)

In [32]:
class LinearArrayApp:
    """
    A GUI for plotting the radiation pattern of a linear array.
      - The full domain [-90°..+90°] is always plotted.
      - HPBW is measured from this domain.
      - The initial zoom is set to [-45°, 45°] (can be changed).
      - "Dezoom" and "Rezoom" functions adjust the x-axis limits.
      - "Plot HPBW vs Phase" shows how HPBW changes with phase shift.
      - Detailed beam parameters (main beam direction and secondary lobe gain/direction)
        are shown in the Info section when requested.
    """

    def __init__(self, root):
        self.root = root
        self.root.title("Full-range Summation + Initial Zoom + Dezoom/Rezoom")

        self.plots_dict = {}

        # User input variables.
        self.N_var = tk.StringVar(value="40")
        self.freq_var = tk.StringVar(value="1.2e9")
        # Phase shift now input in degrees.
        self.delta_var = tk.StringVar(value="0.0")
        self.amin_var = tk.StringVar(value="-45.0")  # initial x-limits after plot
        self.amax_var = tk.StringVar(value="45.0")   # can 'dezoom' to [-90..90]
        self.len_var = tk.StringVar(value="4.0")
        self.scale_var = tk.StringVar(value="linear")

        self.plot_counter = 0

        input_frame = ttk.Frame(root, padding="5 5 5 5")
        input_frame.grid(row=0, column=0, sticky="nw")

        plot_frame = ttk.Frame(root, padding="5 5 5 5")
        plot_frame.grid(row=0, column=1, sticky="nsew")

        toolbar_frame = ttk.Frame(root)
        toolbar_frame.grid(row=1, column=1, sticky="ew")

        info_frame = ttk.Frame(root, padding="5 5 5 5")
        info_frame.grid(row=1, column=0, sticky="nw")

        self.root.columnconfigure(1, weight=1)
        self.root.rowconfigure(0, weight=1)

        r = 0
        ttk.Label(input_frame, text="Number of Elements:").grid(row=r, column=0, sticky="w")
        ttk.Entry(input_frame, textvariable=self.N_var, width=8).grid(row=r, column=1, pady=2)

        r += 1
        ttk.Label(input_frame, text="Frequency (Hz):").grid(row=r, column=0, sticky="w")
        ttk.Entry(input_frame, textvariable=self.freq_var, width=8).grid(row=r, column=1, pady=2)

        r += 1
        # Phase shift (deg)
        ttk.Label(input_frame, text="Phase Shift δφ (deg):").grid(row=r, column=0, sticky="w")
        ttk.Entry(input_frame, textvariable=self.delta_var, width=8).grid(row=r, column=1, pady=2)

        r += 1
        ttk.Label(input_frame, text="Init Zoom Min (deg):").grid(row=r, column=0, sticky="w")
        ttk.Entry(input_frame, textvariable=self.amin_var, width=8).grid(row=r, column=1, pady=2)

        r += 1
        ttk.Label(input_frame, text="Init Zoom Max (deg):").grid(row=r, column=0, sticky="w")
        ttk.Entry(input_frame, textvariable=self.amax_var, width=8).grid(row=r, column=1, pady=2)

        r += 1
        ttk.Label(input_frame, text="Array Length (m):").grid(row=r, column=0, sticky="w")
        ttk.Entry(input_frame, textvariable=self.len_var, width=8).grid(row=r, column=1, pady=2)

        r += 1
        ttk.Label(input_frame, text="Plot Scale:").grid(row=r, column=0, sticky="w")
        ttk.Radiobutton(input_frame, text="Linear", variable=self.scale_var, value="linear").grid(row=r, column=1, sticky="w")
        r += 1
        ttk.Radiobutton(input_frame, text="dB", variable=self.scale_var, value="db").grid(row=r, column=1, sticky="w")

        r += 1
        add_btn = ttk.Button(input_frame, text="Add Plot", command=self.on_add_plot)
        add_btn.grid(row=r, column=0, columnspan=2, pady=5)

        r += 1
        self.plot_combo = ttk.Combobox(input_frame, state="readonly")
        self.plot_combo.grid(row=r, column=0, columnspan=2, sticky="we", pady=5)

        r += 1
        remove_btn = ttk.Button(input_frame, text="Remove Plot", command=self.on_remove_plot)
        remove_btn.grid(row=r, column=0, sticky="ew", pady=3)
        info_btn = ttk.Button(input_frame, text="View Info", command=self.on_view_info)
        info_btn.grid(row=r, column=1, sticky="ew", pady=3)

        r += 1
        clear_btn = ttk.Button(input_frame, text="Clear All", command=self.on_clear_plots)
        clear_btn.grid(row=r, column=0, columnspan=2, pady=5)

        r += 1
        phase_btn = ttk.Button(input_frame, text="Plot HPBW vs Phase", command=self.on_plot_hpbw_vs_phase)
        phase_btn.grid(row=r, column=0, columnspan=2, pady=5)

        r += 1
        dezoom_btn = ttk.Button(input_frame, text="Dezoom [-90..90]", command=self.on_dezoom)
        dezoom_btn.grid(row=r, column=0, columnspan=2, pady=5)

        r += 1
        rezoom_btn = ttk.Button(input_frame, text="Rezoom to Above", command=self.on_rezoom)
        rezoom_btn.grid(row=r, column=0, columnspan=2, pady=5)

        self.figure = plt.Figure(figsize=(6, 4), dpi=100)
        self.axis = self.figure.add_subplot(111)
        self.axis.set_title("Radiation Pattern")
        self.axis.set_xlabel("Angle (deg)")
        self.axis.set_ylabel("Normalized Intensity of Radiation")
        self.axis.grid(True)

        self.canvas = FigureCanvasTkAgg(self.figure, master=plot_frame)
        self.canvas_widget = self.canvas.get_tk_widget()
        self.canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.toolbar = NavigationToolbar2Tk(self.canvas, toolbar_frame)
        self.toolbar.update()

        self.results_label = ttk.Label(info_frame, text="", width=60, justify="left")
        self.results_label.grid(row=0, column=0, sticky="nw")

    def on_add_plot(self):
        """
        1) Compute the radiation pattern over -90°..+90°.
        2) Measure HPBW, main beam direction, and secondary lobe gain.
        3) Plot the pattern with red dashed lines at the half-power edges.
        4) The plot legend displays only the phase (deg & rad) and HPBW.
           Detailed beam info (main beam and secondary lobe) is stored and shown via "View Info".
        5) The x-axis limits are set to the user-specified initial zoom.
        """
        try:
            N = int(self.N_var.get())
            freq = float(self.freq_var.get())
            # Read phase shift in degrees, then convert to radians.
            phase_deg = float(self.delta_var.get())
            phase_rad = np.radians(phase_deg)
            amin = float(self.amin_var.get())
            amax = float(self.amax_var.get())
            leng = float(self.len_var.get())
        except ValueError:
            self._set_msg("Check numeric inputs.")
            return

        (full_phi_deg, pattern_norm, hpbw, left_edge, right_edge,
         main_beam_direction, secondary_gain, secondary_angle) = compute_pattern_full(
            N=N,
            freq_hz=freq,
            delta_phi=phase_rad,
            array_len=leng,
            do_square=True
        )

        # Choose scale: linear or dB.
        scale_mode = self.scale_var.get().lower()
        if scale_mode == "db":
            y_data = 10.0 * np.log10(pattern_norm + 1e-12)
            y_label = "Power (dB)"
        else:
            y_data = pattern_norm
            y_label = "Power (normalized)"

        self.plot_counter += 1
        # Plot legend shows only phase and HPBW.
        plot_label = (f"Graph {self.plot_counter}, δφ={phase_deg:.2f}deg ({phase_rad:.3f}rad), "
                      f"HPBW={hpbw:.2f}°")
        line_main = self.axis.plot(full_phi_deg, y_data, label=plot_label)

        left_line, right_line = None, None
        if hpbw > 0:
            left_line = self.axis.axvline(left_edge, color='red', linestyle='--')
            right_line = self.axis.axvline(right_edge, color='red', linestyle='--')

        self.axis.set_title("Radiation Pattern")
        self.axis.set_xlabel("Angle (deg)")
        self.axis.set_ylabel(y_label)
        self.axis.legend()

        # Set x-axis limits as per initial zoom.
        self.axis.set_xlim(amin, amax)
        self.canvas.draw()

        lines_list = [line_main[0]]
        if left_line:
            lines_list.append(left_line)
        if right_line:
            lines_list.append(right_line)

        # Save all beam parameters in the plots dictionary.
        self.plots_dict[plot_label] = {
            "lines": lines_list,
            "N": N,
            "freq": freq,
            "phase_deg": phase_deg,
            "phase_rad": phase_rad,
            "hpbw": hpbw,
            "left_edge": left_edge,
            "right_edge": right_edge,
            "main_beam_direction": main_beam_direction,
            "secondary_gain": secondary_gain,
            "secondary_angle": secondary_angle
        }

        val_list = list(self.plot_combo['values'])
        val_list.append(plot_label)
        self.plot_combo['values'] = val_list
        self.plot_combo.current(len(val_list) - 1)

    def on_remove_plot(self):
        sel = self.plot_combo.get().strip()
        if not sel or sel not in self.plots_dict:
            self._set_msg("No valid plot selected.")
            return
        for ln in self.plots_dict[sel]["lines"]:
            ln.remove()
        del self.plots_dict[sel]
        new_vals = list(self.plot_combo['values'])
        if sel in new_vals:
            new_vals.remove(sel)
        self.plot_combo['values'] = new_vals
        if new_vals:
            self.plot_combo.current(0)
        else:
            self.plot_combo.set("")
        self.axis.legend()
        self.canvas.draw()
        self.results_label.config(text="")

    def on_clear_plots(self):
        for lbl, info in self.plots_dict.items():
            for ln in info["lines"]:
                ln.remove()
        self.plots_dict.clear()
        self.plot_combo['values'] = []
        self.plot_combo.set("")
        self.axis.legend()
        self.canvas.draw()
        self.results_label.config(text="")

    def on_view_info(self):
        """
        Displays detailed beam parameters for the selected plot:
          - Number of elements, frequency, phase (deg & rad), HPBW, half-power edges,
            main beam direction, and secondary lobe gain and its angle.
        """
        sel = self.plot_combo.get().strip()
        if not sel or sel not in self.plots_dict:
            self.results_label.config(text="No valid plot selected.")
            return
        data = self.plots_dict[sel]
        msg = (
            f"{sel}\n"
            f"N = {data['N']}, freq = {data['freq']:.2e}\n"
            f"Phase = {data['phase_deg']:.2f} deg ({data['phase_rad']:.3f} rad)\n"
            f"HPBW = {data['hpbw']:.2f} deg\n"
            f"Left Edge = {data['left_edge']:.2f} deg\n"
            f"Right Edge = {data['right_edge']:.2f} deg\n"
            f"Direction of Max Gain = {data['main_beam_direction']:.2f} deg\n"
            f"Secondary Lobe: Gain = {data['secondary_gain']:.2f} at {data['secondary_angle']:.2f} deg\n"
        )
        self.results_label.config(text=msg)

    def on_plot_hpbw_vs_phase(self):
        """
        Uses the snippet approach (scanning -90°..+90° for the radiation pattern)
        to plot HPBW vs. phase shift. Here, the phase shift range is set from -100° to 100°.
        """
        try:
            N = int(self.N_var.get())
            freq = float(self.freq_var.get())
            leng = float(self.len_var.get())
        except ValueError:
            self._set_msg("Check numeric inputs for N, freq, length.")
            return

        c = 3e8
        lam = c / freq
        d = (leng / (N - 1)) if N > 1 else 0.0

        # Phase shift range from -100° to 100°.
        phase_shifts_deg = np.linspace(-100, 100, 100)
        hpbw_list = []
        for deg_val in phase_shifts_deg:
            rad_val = np.radians(deg_val)
            bw = snippet_compute_beamwidth(N, d, lam, rad_val)
            hpbw_list.append(bw)

        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(phase_shifts_deg, hpbw_list, 'r-')
        ax.set_xlabel("Phase Shift (deg)")
        ax.set_ylabel("Half-Power Beamwidth (deg)")
        ax.set_title("HPBW vs Phase (Snippet, angles -100°..+100°)")
        ax.grid(True)
        fig.tight_layout()
        plt.show()

    def on_dezoom(self):
        """Sets x-axis limits to [-90, 90]."""
        self.axis.set_xlim(-90, 90)
        self.canvas.draw()

    def on_rezoom(self):
        """Resets x-axis limits to the user-specified initial zoom."""
        try:
            amin = float(self.amin_var.get())
            amax = float(self.amax_var.get())
        except ValueError:
            self._set_msg("Check numeric inputs for rezoom domain.")
            return
        self.axis.set_xlim(amin, amax)
        self.canvas.draw()

    def _set_msg(self, msg):
        self.results_label.config(text=msg)

3.4) Calling the main() function to run the code

In [ ]:
def main3():
    root = tk.Tk()
    app = LinearArrayApp(root)
    root.mainloop()

main3()

3.5) (Step 3) The following code simply plots the beamwidth as a function of phase shift, with the function defined snippet_compute_beamwidth() defined in a previous cell, it is an important plot since it shows how both quantities are related to each other.

In [34]:
# Define fixed parameters
L = 4.0 
wavelength = 0.25  
phase_shifts = np.linspace(-90, 90, 100) 
phase_shifts_rad = np.radians(phase_shifts) 

# Allow user to adjust N dynamically
N_values = [40]

plt.figure(figsize=(8, 6))
for N in N_values:
    d = L / (N - 1)
    hpbw_values = [snippet_compute_beamwidth(N, d, wavelength, delta_phi_rad) for delta_phi_rad in phase_shifts_rad]
    plt.plot(phase_shifts_rad, hpbw_values, 'r', label=f"N = {N}")

plt.xlabel("Phase Shift (Rad)")
plt.ylabel("Half-Power Beamwidth (Degrees)")
plt.title("Beamwidth as a Function of Phase Shift")
plt.legend()
plt.grid(True)
plt.show()

4) (Step 4) In this step, we are required to optimize the number of bins, you will see that we used a binary search approach to perform such an optimization, by adjusting the left and right edges of the beam at each iteration. It is a key function since it provides the number of bins required for a given amount of radiating elements.

4.1) This function generates an amplitude window of uniform weights (all ones) for a specified number of elements, returning an empty array if the number of elements is less than 1.

In [35]:
def amplitude_window(num_elements, window_type="uniform"):
    if num_elements < 1:
        return np.array([])
    return np.ones(num_elements)

4.2) The following function computes the array factor, defined as the radiation intensity's square root. It utlizes the same approach as the one used in the function compute_full_pattern() in Step 3.

In [36]:
def compute_array_factor(N, freq, c, angle_min, angle_max, global_phase_deg,
                         angle_step=0.2, steer_angle_deg=None):
    
    # Calculate spacing and constant parameters
    d = 4.0 / (N - 1)
    lam = c / freq
    k = 2.0 * np.pi / lam

    # Use uniform amplitude weights
    w = amplitude_window(N, "uniform")

    # Create the angle array for the computation domain
    phi_deg = np.arange(angle_min, angle_max + angle_step, angle_step)
    phi_rad = np.radians(phi_deg)

    # Convert the global phase shift to radians
    delta_phase_rad = np.radians(global_phase_deg)

    # Compute the array factor (AF) by summing contributions from each element
    AF = np.zeros_like(phi_rad, dtype=complex)
    for i, ar in enumerate(phi_rad):
        beta = k * d * np.sin(ar) + delta_phase_rad
        accum = 0+0.j
        for n in range(N):
            accum += w[n] * np.exp(1j * n * beta)
        AF[i] = accum

    patt_unnorm = np.abs(AF)**2
    peak_val = max(np.max(patt_unnorm), 1e-12)
    patt_norm = patt_unnorm / peak_val

    # Determine main beam direction from the computed pattern
    idx_peak = np.argmax(patt_norm)
    main_dir = phi_deg[idx_peak]

    # Use the high-resolution beamwidth calculation (which expects phase in radians)
    (_, _, hpbw, left_edge, right_edge,
     main_dir_new, secondary_gain, secondary_angle) = compute_pattern_full(
         N, freq, delta_phase_rad, 4.0, do_square=True
     )

    info = {
        'HPBW': hpbw,
        'left_3db': left_edge,
        'right_3db': right_edge,
        'global_phase': global_phase_deg,
        'main_dir': main_dir_new,
        'secondary_gain': secondary_gain,
        'secondary_angle': secondary_angle
    }
    return phi_deg, patt_norm, info

4.3) The following function simply converts steering angle (deg) into the corresponding/required phase shift (deg).

In [37]:
def compute_phase_for_steerAngle(N, freq, c, steer_deg):
    lam = c / freq
    d = 4.0 / (N - 1)
    k = 2.0 * np.pi / lam
    phase_deg = -np.degrees(k * d * np.sin(np.radians(steer_deg)))
    return phase_deg

4.4) The following function is the core of Step 4 code, it utilizes a binary search approach within the interval [angle_min, angle_max] to find a steering angle whose main-lobe left -3 dB edge is approximately equal to left_edge_target (see report for more details).

In [38]:
def find_steering_for_leftEdge(left_edge_target, N, freq, c,
                               angle_min, angle_max, angle_step=0.2, max_iter=40):
    
    lo = angle_min
    hi = angle_max
    best_steer = 0.5 * (lo + hi)

    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        phase_deg = compute_phase_for_steerAngle(N, freq, c, mid)
        _, _, info = compute_array_factor(
            N, freq, c, angle_min, angle_max, phase_deg,
            angle_step=angle_step, steer_angle_deg=mid
        )
        left_3db = info['left_3db']

        if left_3db > left_edge_target:
            hi = mid
        else:
            lo = mid

        best_steer = 0.5 * (lo + hi)
        if abs(left_3db - left_edge_target) < 1e-4 or (hi - lo) < 1e-4:
            break

    return best_steer

4.5) The following function makes sure to tile the angular domain [angle_min, angle_max] with lobes such that each new lobe's
    left -3 dB edge coincides with the previous lobe's right -3 dB edge (no gap at -3 dB). It returns a list of parameters.

In [39]:
def tile_no_gap(N=40, freq=1.2e9, c=3e8, angle_min=-45.0, angle_max=45.0, angle_step=0.2):
    
    coverage_left = angle_min
    lobes = []
    margin = 2.0

    while coverage_left < (angle_max - margin):
        new_steer = find_steering_for_leftEdge(coverage_left, N, freq, c, angle_min, angle_max, angle_step=angle_step)
        new_phase = compute_phase_for_steerAngle(N, freq, c, new_steer)
        _, _, info = compute_array_factor(
            N, freq, c, angle_min, angle_max, new_phase,
            angle_step=angle_step, steer_angle_deg=new_steer
        )
        L3 = info['left_3db']
        R3 = info['right_3db']
        W3 = info['HPBW']

        if R3 <= coverage_left + 1e-5:
            break

        lobes.append((new_steer, new_phase, W3, L3, R3))
        coverage_left = R3
        if len(lobes) > 300:
            break

    if coverage_left < angle_max:
        new_steer = find_steering_for_leftEdge(coverage_left, N, freq, c, angle_min, angle_max, angle_step=angle_step)
        new_phase = compute_phase_for_steerAngle(N, freq, c, new_steer)
        _, _, info = compute_array_factor(
            N, freq, c, angle_min, angle_max, new_phase,
            angle_step=angle_step, steer_angle_deg=new_steer
        )
        L3 = info['left_3db']
        R3 = info['right_3db']
        W3 = info['HPBW']
        if R3 > coverage_left + 1e-5:
            lobes.append((new_steer, new_phase, W3, L3, R3))

    return lobes

4.6) Main function to execute the complete code.

In [ ]:
def main4():
    # User parameters
    N = 40
    freq = 1.2e9
    c = 3e8
    angle_min = -45.0
    angle_max = 45.0
    angle_step = 0.2

    # Determine lobe tiling (no-gap coverage)
    print("\n== Determining no-gap coverage over the domain ==")
    final_lobes = tile_no_gap(N, freq, c, angle_min, angle_max, angle_step=angle_step)
    n_lobes = len(final_lobes)

    print("SteerAngle (deg) | PhaseShift (deg) | HPBW (deg)       | Left_3dB (deg)   | Right_3dB (deg)")
    for (steerA, phaseA, w3, L3, R3) in final_lobes:
        # Print with full precision
        print(f"{steerA:9.3f}      | {phaseA:9.3f}       | {w3:14.6f}   | {L3:14.6f} | {R3:14.6f}")

    # Plot the dB patterns for each lobe
    plt.figure(figsize=(8, 5))
    for (steerDeg, phaseDeg, HPBW, L3dB, R3dB) in final_lobes:
        phi_deg, patt_norm, info = compute_array_factor(
            N, freq, c, angle_min, angle_max,
            phaseDeg, angle_step=angle_step, steer_angle_deg=steerDeg
        )
        patt_dB = 10.0 * np.log10(np.maximum(patt_norm, 1e-12))
        #labelStr = f"S={steerDeg:.1f}°, Ph={phaseDeg:.1f}°, HPBW={HPBW:.6f}°"
        plt.plot(phi_deg, patt_dB)

    plt.axhline(-3.0, color='red', linestyle='--', label='-3 dB line')
    plt.xlim(angle_min, angle_max)
    plt.ylim(-25, 2)
    plt.xlabel("Azimuth Angle (deg)")
    plt.ylabel("Normalized Power (dB)")
    plt.title(f"No-gap Tiling => {n_lobes} Lobes (Main-lobe Patterns)")
    plt.grid(True)
    plt.legend(fontsize="small")
    plt.show()

    # Plot Phase Shift vs. Steering Angle
    steerAngles = [l[0] for l in final_lobes]
    phaseShifts = [l[1] for l in final_lobes]

    plt.figure(figsize=(6, 5))
    plt.plot(steerAngles, phaseShifts, 'bo-', markersize=6, linewidth=1.5)
    plt.xlabel("Steering Angle (deg)")
    plt.ylabel("Global Phase Shift (deg)")
    plt.title("Phase Shift vs. Steering Angle")
    plt.grid(True)
    plt.show()

    print("\n** Done. Two final plots: Radiation Pattern (in dB scale) & Phase Shift vs. Steering Angle. **\n")
    print(f"\nFound {n_lobes} lobes over the domain.\n")

main4()


== Determining no-gap coverage over the domain ==
SteerAngle (deg) | PhaseShift (deg) | HPBW (deg)       | Left_3dB (deg)   | Right_3dB (deg)
  -42.885      |   100.508       |       4.186047   |     -45.033758 |     -40.847712
  -38.760      |    92.464       |       3.915979   |     -40.757689 |     -36.841710
  -34.915      |    84.532       |       3.735934   |     -36.796699 |     -33.060765
  -31.223      |    76.560       |       3.555889   |     -33.015754 |     -29.459865
  -27.708      |    68.671       |       3.465866   |     -29.459865 |     -25.993998
  -24.280      |    60.730       |       3.330833   |     -25.948987 |     -22.618155
  -20.940      |    52.784       |       3.240810   |     -22.573143 |     -19.332333
  -17.688      |    44.874       |       3.195799   |     -19.287322 |     -16.091523
  -14.458      |    36.874       |       3.150788   |     -16.046512 |     -12.895724
  -11.294      |    28.924       |       3.105776   |     -12.850713 |      -9.7449

7) (Step 7) The following Python code computes the number of angular bins required for a linear array with varying numbers of radiating elements, ranging from 1 to 40. For each value of N , the algorithm determines the number of bins by tiling the full angular domain [−45◦, 45◦] without overlaps and gaps at the -3 dB points of the main lobe, which is done using the same function from Step 4’s.

7.1) The following function converts a desired steering angle (deg) into the required global phase shift (deg)

In [41]:
def compute_phase_for_steerAngle(N, freq, c, steer_deg):
    if N == 1:
        # For a single element, phase shift is irrelevant
        return 0.0

    lam = c / freq
    d = 4.0 / (N - 1)
    k = 2.0 * np.pi / lam
    phase_deg = -np.degrees(k * d * np.sin(np.radians(steer_deg)))
    return phase_deg

7.2) The following function tiles the angular domain [angle_min, angle_max] with lobes such that each new lobe's
    left -3 dB edge coincides with the previous lobe's right -3 dB edge (no gap at -3 dB). This function is similar to the one defined in Step 4 (actually almost the same).

In [42]:
def tile_no_gap(N=40, freq=1.2e9, c=3e8, angle_min=-45.0, angle_max=45.0, angle_step=0.2):
    if N == 1:
        # For a single element, the entire range is covered by one lobe
        return [(0.0, 0.0, 180.0, angle_min, angle_max)]

    coverage_left = angle_min
    lobes = []
    margin = 2.0

    while coverage_left < (angle_max - margin):
        new_steer = find_steering_for_leftEdge(coverage_left, N, freq, c, angle_min, angle_max, angle_step=angle_step)
        new_phase = compute_phase_for_steerAngle(N, freq, c, new_steer)
        _, _, info = compute_array_factor(
            N, freq, c, angle_min, angle_max, new_phase,
            angle_step=angle_step, steer_angle_deg=new_steer
        )
        L3 = info['left_3db']
        R3 = info['right_3db']
        W3 = info['HPBW']

        if R3 <= coverage_left + 1e-5:
            break

        lobes.append((new_steer, new_phase, W3, L3, R3))
        coverage_left = R3
        if len(lobes) > 300:
            break

    if coverage_left < angle_max:
        new_steer = find_steering_for_leftEdge(coverage_left, N, freq, c, angle_min, angle_max, angle_step=angle_step)
        new_phase = compute_phase_for_steerAngle(N, freq, c, new_steer)
        _, _, info = compute_array_factor(
            N, freq, c, angle_min, angle_max, new_phase,
            angle_step=angle_step, steer_angle_deg=new_steer
        )
        L3 = info['left_3db']
        R3 = info['right_3db']
        W3 = info['HPBW']
        if R3 > coverage_left + 1e-5:
            lobes.append((new_steer, new_phase, W3, L3, R3))

    return lobes


7.3) The following function calculates the number of bins required for no-gap coverage for each N (number of elements)
    from 1 to max_elements (inclusive). It returns an array storing the number of bins for each N.

In [43]:
def calculate_bins_for_all_elements(max_elements=40, freq=1.2e9, c=3e8, array_len=4.0, angle_min=-45.0, angle_max=45.0):
    # Array to store bins for each N
    bins_array = []

    for N in range(1, max_elements + 1):
        try:
            # Calculate lobe tiling for the current N
            final_lobes = tile_no_gap(
                N=N,
                freq=freq,
                c=c,
                angle_min=angle_min,
                angle_max=angle_max
            )
            # Count the number of lobes (bins)
            bins_array.append(len(final_lobes))
        except ValueError as e:
            # Handle cases where invalid configurations occur
            bins_array.append(None)
            print(f"Error for N={N}: {e}")

    return np.array(bins_array)

7.4) Main function to run the complete code (takes time to be ran, about 5min).

In [50]:
def main7():
    # Constants
    max_elements = 40
    freq = 1.2e9  # Frequency (Hz)
    c = 3e8  # Speed of light (m/s)
    array_len = 4.0  # Array length (meters)
    angle_min = -45.0  # Min coverage angle (degrees)
    angle_max = 45.0  # Max coverage angle (degrees)

    # Calculate bins for each N
    bins_array = calculate_bins_for_all_elements(
        max_elements=max_elements,
        freq=freq,
        c=c,
        array_len=array_len,
        angle_min=angle_min,
        angle_max=angle_max
    )

    # Print results
    print("Number of bins for each N (1 to 40):")
    for N, bins in enumerate(bins_array, start=1):
        print(f"N = {N}: bins = {bins}")

    # Plot results
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, max_elements + 1), bins_array, marker="o", label="Bins vs. Number of Elements")
    plt.xlabel("Number of Radiating Elements (N)")
    plt.ylabel("Number of Bins")
    plt.title("Number of Bins vs. Number of Elements")
    plt.grid(True)
    plt.legend()
    plt.show()

main7()

Number of bins for each N (1 to 40):
N = 1: bins = 1
N = 2: bins = 1
N = 3: bins = 1
N = 4: bins = 1
N = 5: bins = 1
N = 6: bins = 1
N = 7: bins = 1
N = 8: bins = 2
N = 9: bins = 1
N = 10: bins = 1
N = 11: bins = 2
N = 12: bins = 1
N = 13: bins = 1
N = 14: bins = 1
N = 15: bins = 2
N = 16: bins = 2
N = 17: bins = 2
N = 18: bins = 2
N = 19: bins = 2
N = 20: bins = 2
N = 21: bins = 2
N = 22: bins = 2
N = 23: bins = 1
N = 24: bins = 1
N = 25: bins = 1
N = 26: bins = 1
N = 27: bins = 1
N = 28: bins = 1
N = 29: bins = 26
N = 30: bins = 26
N = 31: bins = 27
N = 32: bins = 26
N = 33: bins = 26
N = 34: bins = 26
N = 35: bins = 26
N = 36: bins = 26
N = 37: bins = 27
N = 38: bins = 27
N = 39: bins = 27
N = 40: bins = 27


You can see in the results above that the first number of radiating elements for which the number of bins is valid is 29, which matches our theoretical value!

8) In Step 8, we are asked to find a way to reduce the secondary lobe gains, in the expression of radiation intensity, we can define what we call windows (Hamming, Hann, Uniform...). These windows are specific ways to define tapering weights within the DTFT expression, with values of alpha and beta (see function 8.1)) specific to these windows. 

Useful global variables (variables which aren't defined locally within functions):

In [44]:
f = 1200e6 
c = 3e8 
lambda_ = c / f  
k = 2 * np.pi / lambda_  
angles = np.linspace(-90, 90, 1000)  
L = 4  
N = 29  #Optimized number of elements chosen

8.1) The following function defines the tapering weights depending on the selected window.

In [45]:
def taper_weights(N, taper_type):
    n = np.arange(N)
    if taper_type == "hamming":
        #Usual values for this window
        alpha = 0.54
        beta = 0.46
        return alpha - beta * np.cos((2 * np.pi * n) / (N - 1))
    elif taper_type == "hann":
        #Usual values for this window
        alpha = 0.5
        beta = 0.5
        return alpha - beta * np.cos((2 * np.pi * n) / (N - 1))
    elif taper_type == "uniform":  
        #Uniform weights: E_0 = 1
        return np.ones(N)

8.2) The following function defines a way to compute the radiation intensity, in a similar fashion of the one defined in Step 3 (compute_pattern_full()).

In [46]:
# Function to calculate radiation intensity (I = AF^2)
def radiation_intensity(theta, N, taper_type):
    d = L / (N - 1)  # Element spacing
    theta_rad = np.radians(theta)
    big_delta_phi = k * d * np.sin(theta_rad) 
    
    # Generate weights for the selected tapering method
    weights = np.array(taper_weights(N, taper_type))[:, np.newaxis]
    
    # Radiation intensity
    element_indices = np.arange(N)
    element_phase = np.exp(1j * element_indices[:, np.newaxis] * big_delta_phi)
    numerator = np.sum(weights * element_phase, axis=0) 
    I = (np.abs(numerator) / np.max(np.abs(numerator)))**2

    return I

8.3) Main code to plot radiation intensity for different windows.

In [47]:
if __name__ == "__main__":
    # Compute intensity for all tapers
    I_uniform = radiation_intensity(angles, N, "uniform")
    I_hamming = radiation_intensity(angles, N, "hamming")
    I_hann = radiation_intensity(angles, N, "hann")

    # Plot results
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(angles, 10 * np.log10(I_uniform), label="Uniform", color="blue", linewidth=2)
    ax.plot(angles, 10 * np.log10(I_hamming), label="Hamming", color="green", linewidth=2)
    ax.plot(angles, 10 * np.log10(I_hann), label="Hann", color="red", linewidth=2)

    # Formatting
    ax.set_title("Radiation Intensity for Uniform, Hamming, and Hann Windows")
    ax.set_xlabel("Azimuth Angle (degrees)")
    ax.set_ylabel("Radiation Intensity (dB)")
    ax.set_xlim(-90, 90)
    ax.set_ylim(-50, 10)
    ax.grid(True)
    ax.legend(loc="upper right", fontsize=10)

    plt.show()